# Web Scraping com BeautifulSoup

#### <strong>Introdução</strong>
Este projeto foi realizado por Gabriel Will, a fim de estudos.<br>
Site: <strong><a href="https://gabrielwill.com.br">gabrielwill.com.br</a></strong><br>
Linkedin: <strong><a href="https://www.linkedin.com/in/gabrielwillgg/">Gabriel Will</a></strong>

#### <strong>Objetivo</strong>
O objetivo deste notebook é desenvolver um web scraping, utilizando BeautifulSoup, do site CoinMarketCap, com o intuito de retornar o top 10 criptomoedas com maiores MarketCaps da atualidade.

Importamos dos pacotes que usaremos para desenvolver este projeto, sendo elas:
- BeautifulSoup: o pacote principal do projeto, usado para extrair os dados da página.
- Requests: a biblioteca essencial para podermos enviar uma requisição HTTP para o destino e recebermos o conteúdo do site como resposta.
- RegEx(re): um módulo que vamos usar para podermos efetuar buscas de caracteres ou textos específicos.
- Pandas: poderosíssima biblioteca para podermos gerar um DataFrame e visualizar o resultado de nosso web scraping.

In [1]:
# Importando pacotes
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# Realizando a requisição no site do CoinMarketCap
link = 'https://coinmarketcap.com/'
request = requests.get(link)
site = BeautifulSoup(request.text,"html.parser")

Analisando o site podemos perceber que a tabela com a lista de criptomoedas está envolta por uma tag _tbody_, onde cada linha dessa coluna é um _tr_.

In [3]:
# Coletando os dados da tabela e separando as linhas
tbody = site.find("tbody")
rows = tbody.find_all("tr")

# Declaração do dicionário final, onde estará reúnido todas as cryptos.
cryptos = {}

In [4]:
# Percorrendo cada linha da lista de linhas para podermos separar por crypto.
for row in rows:
    # try:except para poder tratar um erro de atributo, devido o BeautifulSoup não conseguir ter acesso a todas as cryptos da tabela. Quando ele passa da linha 10, ele encerra o <for>.
    try:
        # retornando todos os valores com cifrão (Price, Market Cap e Volume)
        values = row.find_all(string=re.compile(r"\$"))
        # tratando o valor da porcentagem para saber se é um valor negativo ou não.
        percentages = row.find_all(string=re.compile("%")) 

        # tratando o valor da porcentagem para saber se é um valor negativo ou não.
        for i, percentage in enumerate(percentages): 
            if "iEhQde" in percentage.parent["class"]:
                percentages[i] = "-" + percentage

        name = row.find(class_="kKpPOn").text
        code = row.find(class_="coin-item-symbol").text
        price = values[0]
        var_1h = percentages[0]
        var_24h = percentages[1]
        var_7d = percentages[2]
        market_cap = values[2]
        volume = values[3]

        # criação do dicionário da crypto referente a linha atual
        new_crypto = {"Code": code, "Price": price,"Var 1h": var_1h, "Var 24h": var_24h, "Var 7d": var_7d, "Market Cap": market_cap, "Volume": volume}

        # adicionando a crypto no dicionário geral das cryptos
        cryptos[name] = new_crypto

    except AttributeError:
        break

In [5]:
# Gerando um DataFrame para melhor visualização do resultado do Web Scraping.
cryptos_df = pd.DataFrame(cryptos).T
cryptos_df.index.names = ["Name"]
display(cryptos_df)

,Code,Price,Var 1h,Var 24h,Var 7d,Market Cap,Volume
Name,,,,,,,
Bitcoin,BTC,"$63,186.50",-0.91%,-1.32%,-0.31%,"$1,244,791,090,207","$24,489,655,775"
Ethereum,ETH,"$3,109.69",-0.70%,-0.74%,2.61%,"$379,362,012,179","$10,732,319,891"
Tether,USDT,$0.9998,0.01%,-0.03%,-0.08%,"$110,403,382,253","$45,483,721,067"
BNB,BNB,$593.34,-0.81%,-3.17%,7.78%,"$87,466,239,603","$824,772,332"
Solana,SOL,$137.03,-1.84%,-4.12%,-2.29%,"$61,271,732,711","$2,528,531,037"
USDC,USDC,$1.00,0.00%,0.00%,-0.01%,"$33,393,322,815","$6,272,772,739"
XRP,XRP,$0.519,-1.28%,-0.46%,3.90%,"$28,669,539,764","$1,141,796,349"
Dogecoin,DOGE,$0.1467,-1.21%,-1.56%,-4.41%,"$21,125,050,522","$873,874,041"
Toncoin,TON,$5.27,-1.18%,-1.69%,-11.19%,"$18,315,606,559","$166,358,658"
